## Import

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# For reproducibility
torch.manual_seed(1)

## Discrete Probability Distribution

## Softmax

가위바위보를 하는 상황을 가정해보자.<br>
상대방이 가위를 냈을 때 다음에 무엇을 낼지 예측하려면,<br>
$P(주먹|가위) P(가위|가위) P(보|가위)$ 를 알면 된다.

$P(class = i)={e^{i}\over \sum e^{i}}$

In [4]:
z = torch.FloatTensor([1, 2, 3]) # max = (0, 0, 1)

In [5]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


위의 가정에서 $P(보|가위)$가 0.6652가 된다.

In [7]:
# 출력값의 합은 항상 1이다.
hypothesis.sum()

tensor(1.)

## Cross Entropy

$H(P, Q)= -\mathbb{E}_{x\sim P(x)}[logQ(x)]=-\sum_{x\in \chi}P(x)logQ(x)$

## Cross Entropy Loss(Low-level)

$L={1\over{N}} \sum -ylog(\hat{y})$<br><br>
이때 $y=P(x)$, $\hat{y}=Q(x)$

In [11]:
z = torch.rand(3, 5, requires_grad=True) # |z|=(3, 5)
hypothesis = F.softmax(z, dim=1) # 2행에 대해서 수행해라
print(hypothesis)
# prediction = y햇이 됨

tensor([[0.2570, 0.1524, 0.2303, 0.1721, 0.1882],
        [0.1178, 0.1501, 0.2499, 0.1687, 0.3135],
        [0.2615, 0.1711, 0.1254, 0.2375, 0.2045]], grad_fn=<SoftmaxBackward>)


In [64]:
# 랜덤하게 정답생성
# classes = 5
# samples = 3
y = torch.randint(5, (3,)).long()
print(y)
# 인덱스값임

tensor([3, 1, 2])


In [66]:
y_one_hot = torch.zeros_like(hypothesis) # |y_one_hot| = (3, 5)
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # dim=1에서 y.unsqueeze(1)을 해가지고 1을 뿌려라
# |y|=(3,) => (3, 1)
# [3, 1, 2] => [3
#               1
#               2]

tensor([[0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [67]:
#1 step (3, 5) * (3, 5)= (3, 5) 
#2 step (3, 5)-> (3,)
#3 step (3,)->scalar
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.9108, grad_fn=<MeanBackward0>)


## Cross-entropy Loss with torch.nn.functional

In [70]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.9108, grad_fn=<MeanBackward0>)

In [71]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y)
# nll = negative log likelihood

tensor(1.9108, grad_fn=<NllLossBackward>)

In [72]:
# more High level
F.cross_entropy(z, y)

tensor(1.9108, grad_fn=<NllLossBackward>)

## Training with Low-level Cross Entropy Loss

In [73]:
# |x_train| = (m, 4)
x_train = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7],]
# |y_train| = (m,)
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train) # 이산함수를 근사하기 위해 정수로 바꿈

In [75]:
# 모델 초기화

# samples = m
# classes = 3
# dim = 4 
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


## Training with F.cross_entropy

In [76]:
# 모델 초기화

W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # cost 계산 (2)
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## High-level Implementation with nn.Module

In [80]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # 4개의 입력받아서 3개의 클래스에 대한 각각의 확률값 구해줌
        
    def forward(self, x): # |x| = (m, 4) => |prediction| = (m, 3)
        return self.linear(x)

In [78]:
model = SoftmaxClassifierModel()

In [79]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.cross_entropy(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.113673
Epoch  100/1000 Cost: 0.714719
Epoch  200/1000 Cost: 0.638072
Epoch  300/1000 Cost: 0.581891
Epoch  400/1000 Cost: 0.532020
Epoch  500/1000 Cost: 0.484810
Epoch  600/1000 Cost: 0.438817
Epoch  700/1000 Cost: 0.393323
Epoch  800/1000 Cost: 0.347979
Epoch  900/1000 Cost: 0.302978
Epoch 1000/1000 Cost: 0.261105
